# Tonga 5m DEM creation
Load in LiDAR where it exists and combine with FABDEM to produce 5m DEMs

In [3]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pathlib

import os
import sys
module_path = os.path.abspath(os.path.join("..", "src", "pacific-dems"))
if module_path not in sys.path:
    sys.path.append(module_path)

import create_dem_functions
import islands
import create_data_paths

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Define country and resolution

In [4]:
resolution = 5 # in meters
country_name = "tonga"

# Create paths and get contry information

In [5]:
paths = create_data_paths.get_paths(country_name=country_name, resolution=resolution)
islands_dict = islands.get_island(country_name)

# Specify any LiDAR data

In [6]:
# Path of LiDAR DEMs
eua_lidar = create_dem_functions.load_dem(paths["lidar"] / "Eua_1m_topobathy.tif") 
tafahi_lidar = create_dem_functions.load_dem(paths["lidar"] / "Tafahi_1m_topobathy.tif") 
vavau_lidar = create_dem_functions.load_dem(paths["lidar"] / "Vavau_1m_topobathy.tif") 

tongatapu_lidar_land = create_dem_functions.load_dem(paths["lidar"] / "TongaLidar2011_1m_dem_UTM_compressed.tif") 
tongatapu_lidar_bathy = create_dem_functions.load_dem(paths["lidar"] / "TongaLidar2011_5m_Bathymetry_UTM_compressed.tif") 
lifuka_lidar_land = create_dem_functions.load_dem(paths["lidar"] / "LifukaLidar2011_1m_DEM_UTM_compressed.tif") 
lifuka_lidar_bathy = create_dem_functions.load_dem(paths["lidar"] / "LifukaLidar2011_5m_Bathymetry_UTM_compressed.tif") 


# Add to islands
islands_dict["'eua"]["lidar"] = eua_lidar
islands_dict["niuatoputapu_tafahi_islands"]["lidar"] = tafahi_lidar
islands_dict["vava'u_group"]["lidar"] = vavau_lidar
islands_dict["tongatapu"]["lidar"] = [tongatapu_lidar_land, tongatapu_lidar_bathy]
islands_dict["ha'apai_group"]["lidar"] = [lifuka_lidar_land, lifuka_lidar_bathy]

# Get island outlines and combine FABDEMs

In [7]:
islands_dict.pop("minerva") # remove as no FABDEM
for island_name, island_dict in islands_dict.items():
    print(f"Setup for {island_name}")
    island_dict["name"] = island_name
    island_dict["boundary"] = create_dem_functions.create_boundary_epsg4326(island_dict=island_dict, output_path=paths["output"])
    island_dict["fab"] = create_dem_functions.combine_fabs_in_boundary(island_dict=island_dict, fab_path=paths["fabdem"])

Setup for 'ata
Setup for 'eua
Setup for tongatapu
Setup for hunga_tonga-hunga_ha'apai_islands
Setup for ha'apai_group
Setup for vava'u_group
Setup for niuatoputapu_tafahi_islands
Setup for niuafo'ou


# Create DEMs over islands

In [8]:
lidar_only_island_dict = {"tongatapu": islands_dict.pop("tongatapu")}

In [ ]:
create_dem_functions.loop_through_islands_creating_dems(island_groups=islands_dict,
                                                        resolution=resolution, output_path=paths["output"])

In [ ]:
create_dem_functions.loop_through_islands_creating_dems_from_lidar_only(island_groups=lidar_only_island_dict,
                                                                        resolution=resolution, output_path=paths["output"],
                                                                        interpolate=True)

Resampling DEMs for tongatapu to 5m.
	Combining [<xarray.DataArray (y: 25000, x: 35000)>
dask.array<getitem, shape=(25000, 35000), dtype=float32, chunksize=(958, 35000), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float32 6.7e+05 6.7e+05 6.7e+05 ... 7.05e+05 7.05e+05
  * y            (y) float32 7.671e+06 7.671e+06 ... 7.646e+06 7.646e+06
    spatial_ref  int32 0
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_MAXIMUM:        64.789726257324
    STATISTICS_MEAN:           14.179388640771
    STATISTICS_MINIMUM:        -1.1284807920456
    STATISTICS_STDDEV:         12.277000433039
    STATISTICS_APPROXIMATE:    YES
    STATISTICS_VALID_PERCENT:  30.82
    scale_factor:              1.0
    add_offset:                0.0
    _FillValue:                nan, <xarray.DataArray (y: 5000, x: 7600)>
dask.array<getitem, shape=(5000, 7600), dtype=float32, chunksize=(4415, 7600), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float32 6.72e+05 6.72e+05 6.